In [41]:
from transformers import BartTokenizer,BartTokenizerFast,BartForConditionalGeneration
tokenizer = BartTokenizerFast.from_pretrained("facebook/bart-large-xsum")
tokfast = BartTokenizerFast.from_pretrained("facebook/bart-large-xsum")
model = BartForConditionalGeneration.from_pretrained("facebook/bart-large-xsum")

In [42]:
ARTICLE_TO_SUMMARIZE = (
    "PG&E stated it scheduled the blackouts in response to forecasts for high winds "
    "amid dry conditions. The aim is to reduce the risk of wildfires. Nearly 800 thousand customers were "
    "scheduled to be affected by the shutoffs which were expected to last through at least midday tomorrow."
)
inputs = tokenizer([ARTICLE_TO_SUMMARIZE], max_length=1024, return_tensors="pt")

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


In [43]:
import torch

In [44]:

dec_inputs = [[1250, 581, 4871, tokenizer.bos_token_id, 8332, 947, 717, 2305, 24]]
dec_mask = torch.tensor([[0.,0.,0.,1.,1.,1.,1.,1., 1. ]])
dec_input_tensor  = torch.tensor(dec_inputs)
print(inputs['input_ids'].size())
print(dec_input_tensor.size())
output = model(input_ids=inputs['input_ids'], decoder_input_ids = dec_input_tensor,decoder_attention_mask=dec_mask)
logits = output.logits[:, -1, :]

torch.Size([1, 56])
torch.Size([1, 9])


In [45]:
import torch.nn.functional as F
def visualize_logits(logits, raw_indicies, name, tokenizer, idx=0, topk=10):
    # assume logits in batch
    tmp_logit = logits[idx]

    values, indices = torch.topk(tmp_logit, k=topk, dim=-1)  # batch x pretopk
    post_probs = F.softmax(values, dim=-1)
    post_probs_list = post_probs.tolist()
    tmp_raw = raw_indicies[idx]
    out = []
    for i in range(5):
        out.append(str(post_probs_list[i])[:4] + "  "+ tokenizer.decode(tmp_raw[indices[i]]) )
        # out.append( tokenizer.decode(tmp_raw[indices[i]]) )
    print(name + ":\n" + "\n".join(out) + "\n-----------")


batch_size = 1
top_logits, indices = torch.topk(logits, k=200, dim=-1)  # batch x pretopk
post_logits, post_indices = top_logits.topk(5, dim=1)
post_probs = torch.nn.functional.softmax(post_logits, dim=1)
index_into_top_indices = post_indices[
            torch.arange(batch_size).to(post_indices.device), :5
        ]  # batch, k_best
for b in range(batch_size):
    next_indices = indices[b][index_into_top_indices[b]].cpu().tolist() # k best options
    visualize_logits(top_logits, indices,"RAW", tokenizer,b,10)

RAW:
0.33   will
0.13   has
0.11   was
0.10   had
0.08   is
-----------


In [ ]:
from transformers import pipeline, set_seed

set_seed(32)
generator = pipeline('text-generation', model="facebook/opt-1.3b", do_sample=True)
generator("Hello, I'm am conscious and")


In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM

tokenizer = AutoTokenizer.from_pretrained("gpt2")

model = AutoModelForCausalLM.from_pretrained("gpt2")

In [ ]:

tokenizer = AutoTokenizer.from_pretrained("gpt2")
model = AutoModelForCausalLM.from_pretrained("gpt2")


In [ ]:

input_ids = tokenizer("name[The Eagle], eatType[coffee shop], food[Japanese], priceRange[less than £20]", return_tensors="pt")['input_ids']
outputs = model.generate(input_ids, do_sample=False, max_length=100)
print(tokenizer.batch_decode(outputs, skip_special_tokens=True))


print(outputs)

In [ ]:
from transformers import T5ForConditionalGeneration

model = T5ForConditionalGeneration.from_pretrained("google/t5-v1_1-base")

In [ ]:
from transformers import T5Tokenizer, T5ForConditionalGeneration

tokenizer = T5Tokenizer.from_pretrained("t5-small")
model = T5ForConditionalGeneration.from_pretrained("t5-small")


In [ ]:

# inference
input_ids = tokenizer(
    "summarize: name[The Eagle], eatType[coffee shop], food[Japanese], priceRange[less than £20]", return_tensors="pt"
).input_ids  # Batch size 1
outputs = model.generate(input_ids)
print(tokenizer.decode(outputs[0], skip_special_tokens=True))
# studies have shown that owning a dog is good for you.

In [6]:
import os
import pickle
print(os.getcwd())
with open('../vizs/bs_1.05_10_5_40375560.pkl','rb') as fd:
    output_dict = pickle.load(fd)
print(output_dict)


/export/home/cond-text-gen/dec


ModuleNotFoundError: No module named 'dec'